In [2]:
import pandas as pd
from IPython.display import clear_output
import pandas as pd

In [3]:
import time
import random

In [4]:
name_info = pd.read_csv('./corrected_matches.csv')
name_info.head(2)

,ID,Name,School
0,880302,Lauren Johnson,Bryn Mawr
1,1653662,Kyle Johnson,"University of North Carolina, Wilmington"


,Name,School,Power,Link,ID
0,Tiago ASAKAWA,UMBC,15.96,/swimmer/515075,515075
1,Niklas Weigelt,UMBC,8.38,/swimmer/413265,413265


In [5]:
name_info = name_info.set_index('ID')
name_info.head(2)

,Name,School
ID,,
880302,Lauren Johnson,Bryn Mawr
1653662,Kyle Johnson,"University of North Carolina, Wilmington"


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import clear_output

def get_swimmer_events(swimmer_id):
    url = f"https://www.swimcloud.com/swimmer/{swimmer_id}/powerindex/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
        'Referer': 'https://google.com/'
    }
    
    while True:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to retrieve data for swimmer_id {swimmer_id}")
            time.sleep(60)
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='c-table-clean')
        
        if table is None:
            print("Table not found, skipping swimmer...")
            time.sleep(7)
            return None

        rows = table.find('tbody').find_all('tr') if table.find('tbody') else None
        if rows is None:
            print("Rows not found, skipping swimmer...")
            time.sleep(7)
            return None
        
        break

    data = {}
    for row in rows:
        cols = row.find_all('td')
        event = cols[0].text.strip()
        time_HS = cols[1].text.strip()
        points = cols[3].text.strip()
        data[event] = (time_HS, points)
    
    return data

def create_swimmer_dataframe(swimmer_ids):
    events = [
        "50 Y Free", "50 S Free", "50 L Free", "100 Y Free", "100 S Free", "100 L Free", "200 Y Free", "200 S Free", 
        "200 L Free", "500 Y Free", "400 S Free", "400 L Free", "1000 Y Free", "800 S Free", "800 L Free", "1650 Y Free", 
        "1500 S Free", "1500 L Free", "100 Y Back", "100 S Back", "100 L Back", "200 Y Back", "200 S Back", "200 L Back", 
        "100 Y Breast", "100 S Breast", "100 L Breast", "200 Y Breast", "200 S Breast", "200 L Breast", "100 Y Fly", 
        "100 S Fly", "100 L Fly", "200 Y Fly", "200 S Fly", "200 L Fly", "100 Y IM", "100 S IM", "200 Y IM", "200 S IM", 
        "200 L IM", "400 Y IM", "400 S IM", "400 L IM"
    ]
    
    columns = [f"{event} Time" for event in events] + [f"{event} Points" for event in events]
    result_df = pd.DataFrame(index=swimmer_ids, columns=columns)
    
    for i, swimmer_id in enumerate(swimmer_ids, 1):
        swimmer_data = get_swimmer_events(swimmer_id)
        if swimmer_data:
            for event in events:
                time_col = f"{event} Time"
                points_col = f"{event} Points"
                time_HS, points = swimmer_data.get(event, ('', ''))
                result_df.at[swimmer_id, time_col] = time_HS
                result_df.at[swimmer_id, points_col] = points
        
        if i % 10 == 0:
            print(f"Processed {i} swimmers...")

        if i % 100 == 0:  # Save every 100 swimmers
            result_df.to_parquet('progress_snapshot.parquet', engine='pyarrow')
        
        if i % 200 == 0:
            clear_output(wait=True)
            time.sleep(10)

    return result_df

# Load the existing DataFrame
# df = pd.read_parquet('./4more_name_info_dtypes.parquet', engine='pyarrow')  # Adjust the file path as needed
swimmer_ids = name_info.index.tolist()

# Create the new DataFrame
new_df = create_swimmer_dataframe(swimmer_ids)
print("Done")

Table not found, skipping swimmer...
Processed 610 swimmers...
Processed 620 swimmers...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Processed 630 swimmers...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Processed 640 swimmers...
Processed 650 swimmers...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Table not found, skipping swimmer...
Done


In [7]:
new_df

,50 Y Free Time,50 S Free Time,50 L Free Time,100 Y Free Time,100 S Free Time,100 L Free Time,200 Y Free Time,200 S Free Time,200 L Free Time,500 Y Free Time,...,200 S Fly Points,200 L Fly Points,100 Y IM Points,100 S IM Points,200 Y IM Points,200 S IM Points,200 L IM Points,400 Y IM Points,400 S IM Points,400 L IM Points
880302,25.62,,,,,,,,,,...,,,,,47.61,,,,,
1653662,,,26.50,,,56.48,1:48.93,,,4:51.89,...,,,100.00,,,,47.32,55.96,,
1653662,,,26.50,,,56.48,1:48.93,,,4:51.89,...,,,100.00,,,,47.32,55.96,,
1131128,22.77,,,49.20,,,1:48.10,,,4:52.60,...,,,100.00,,38.46,,,83.96,,
483789,22.71,,,51.62,,,,,2:45.24,,...,,,94.93,,100.00,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484075,23.95,,,52.11,,,2:01.39,,,6:21.99,...,,,,,100.00,,,,,
1513876,23.62,,,49.56,,,1:45.84,,,4:47.12,...,,,100.00,,84.46,,,77.39,,
579179,21.90,,,48.59,,,1:53.25,,,5:31.43,...,,,94.75,,77.65,,,,,
880973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
new_df.dropna(how='all',inplace=True)

In [9]:
new_df

,50 Y Free Time,50 S Free Time,50 L Free Time,100 Y Free Time,100 S Free Time,100 L Free Time,200 Y Free Time,200 S Free Time,200 L Free Time,500 Y Free Time,...,200 S Fly Points,200 L Fly Points,100 Y IM Points,100 S IM Points,200 Y IM Points,200 S IM Points,200 L IM Points,400 Y IM Points,400 S IM Points,400 L IM Points
880302,25.62,,,,,,,,,,...,,,,,47.61,,,,,
1653662,,,26.50,,,56.48,1:48.93,,,4:51.89,...,,,100.00,,,,47.32,55.96,,
1653662,,,26.50,,,56.48,1:48.93,,,4:51.89,...,,,100.00,,,,47.32,55.96,,
1131128,22.77,,,49.20,,,1:48.10,,,4:52.60,...,,,100.00,,38.46,,,83.96,,
483789,22.71,,,51.62,,,,,2:45.24,,...,,,94.93,,100.00,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504146,21.66,,,46.32,,,1:40.30,,,4:31.42,...,,,,,15.51,,,34.88,,
504146,21.66,,,46.32,,,1:40.30,,,4:31.42,...,,,,,15.51,,,34.88,,
1484075,23.95,,,52.11,,,2:01.39,,,6:21.99,...,,,,,100.00,,,,,
1513876,23.62,,,49.56,,,1:45.84,,,4:47.12,...,,,100.00,,84.46,,,77.39,,


In [10]:
new_df.to_csv('corrected_HS_Times.csv',index=True)